# Testing File
---
_This notebook is for testing new functionality before introducing it into the program._

### Protocol
---
_Pokémon Showdown is accessible via websockets, so I can interact with the website from this file (sending messages, logging in, and so on)._
_Found here: https://github.com/smogon/pokemon-showdown/blob/master/PROTOCOL.md_

_This has been done before in python here: https://github.com/pmariglia/showdown/blob/master/showdown/websocket_client.py_

**Variables I'll need**:
* Websocket UI
* Username and Password
* Format
* Team to use (if applicable)

In [10]:
import asyncio
import websockets
import requests

uri = "wss://sim3.psim.us/showdown/websocket"
loginuri = "https://play.pokemonshowdown.com/action.php"
testusername = "thisisatest12345"
testpassword = "password" # very secure :)

socket = await websockets.connect(uri)
print(socket)
challid = 0
challstr = ""
i = True


while(i):
    received = await socket.recv()
    print(received)

    messages = received.split("|")
    if(messages[1] == "challstr"):
        challid = messages[2]
        challstr= messages[3]
        i = False

challstr = "|".join([challid, challstr])
print(challstr)


loggedIn = requests.post(
    loginuri,
    data={
        "act": "login",
        "name": testusername,
        "pass": testpassword,
        "challstr": challstr,
    }
)
print(loggedIn.text)



4
701a0c5b9aed9048a32ed575628da8ad35444b7cf1eaaf04fd143d04b37bbdc25a972764a37e8dda6a2de1af41d719ffe3e0a1305d3ce0c4040d9fbaa3f9b68684cc6f2a997b112c55d719778f1e7935938c3540f1360baaa49f97cc1939a1554b9290e4eeaa94d38126e07fe0b94b4f02e33ded2b52919f5f61933ac8072e24
4|701a0c5b9aed9048a32ed575628da8ad35444b7cf1eaaf04fd143d04b37bbdc25a972764a37e8dda6a2de1af41d719ffe3e0a1305d3ce0c4040d9fbaa3f9b68684cc6f2a997b112c55d719778f1e7935938c3540f1360baaa49f97cc1939a1554b9290e4eeaa94d38126e07fe0b94b4f02e33ded2b52919f5f61933ac8072e24
]{"actionsuccess":true,"assertion":"701a0c5b9aed9048a32ed575628da8ad35444b7cf1eaaf04fd143d04b37bbdc25a972764a37e8dda6a2de1af41d719ffe3e0a1305d3ce0c4040d9fbaa3f9b68684cc6f2a997b112c55d719778f1e7935938c3540f1360baaa49f97cc1939a1554b9290e4eeaa94d38126e07fe0b94b4f02e33ded2b52919f5f61933ac8072e24,thisisatest12345,2,1730895984,sim3.psim.us,1f7cb6a3aa4ccea51730895984,b9ccdf14115f885f,;13906bbab871f372808dd2aae81f3f169555b91f5b16bbd206a041f56f07f1c1eef2e793af173d97714734e367ba4d038469a